<a href="https://colab.research.google.com/github/bchuchucaa/RNNConsumoDeAgua/blob/main/RNNConsumoDeAgua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip

unzip:  cannot find or open /content/archive.zip, /content/archive.zip.zip or /content/archive.zip.ZIP.


In [18]:
from pandas import read_csv
from datetime import datetime
import pandas as pd
from matplotlib import pyplot
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
print('LIBRERIAS IMPORTADAS')

LIBRERIAS IMPORTADAS


**LECTURA DE DATOS**

In [34]:
#LOADING DATA 
df= read_csv('/content/AguaH.csv')
df = df.iloc[0:5000 , :]
#AJUSTE DEL FORMATO DE LAS FECHAS PARA LA TRANSFORMACION A TIPO DATETIME
lookup = {'ENE':'01','FEB':'02','MAR':'03','ABR':'04','MAY':'05','JUN':'06','JUL':'07','AGO':'08','SEP':'09','OCT':'10','NOV':'11','DIC':'12'}
clist=[]
for col in df.columns[5:]:
    col = col[4:].split('_')
    clist.append('20'+col[1]+'-'+lookup[col[0]]+'-01')

columns = ['LANDUSE_TYPE','USER','PIPE DIAM','VENDOR','JAN16']+clist
df.columns=columns
df = df.drop(columns=['LANDUSE_TYPE','PIPE DIAM','VENDOR','JAN16'])
#df=df.transpose()
print("Dataframe shape ",df.shape)


Dataframe shape  (5000, 85)


In [20]:
#ESTE CODIGO SIRVE UNICAMENTE PARA LA INTEGRACION DE LOS NOMBRES EN EL DATAFRAME
col_list = ["Name"]
names = pd.read_csv("/content/StateNames.csv", usecols=col_list)
names = names.iloc[0:84 , :]
#ELIMINAMOS LA COLUMNA DE LOS NOMBRE YA QUE SE BINARIZARON
#df = df.iloc[: , 1:]
print("Names shape ",names.shape)

Names shape  (84, 1)


In [61]:
rows_list=[]
dataComp = pd.DataFrame(columns=('DATE','TIPO', 'CONSUMO'))
for column in range(1,len(df.columns),1):
  selection=df.iloc[:,column]
  for row in range(len(selection)):
    df2=[df.columns[column],df.iloc[row,0],selection.iloc[row]]
    rows_list.append(df2)

In [62]:
dataComp = pd.DataFrame(rows_list)               

In [63]:
dataComp.head()

,0,1,2
0,2009-01-01,DOMESTICO MEDIO,20.0
1,2009-01-01,DOMESTICO MEDIO,NaN
2,2009-01-01,DOMESTICO RESIDENCIAL,NaN
3,2009-01-01,DOMESTICO MEDIO,NaN
4,2009-01-01,DOMESTICO RESIDENCIAL,20.0


In [64]:
dataComp.rename(columns = {0:'DATE',1:'TIPO',2:'CONSUMO'}, inplace = True)

In [67]:
dataComp['DATE'] = pd.to_datetime(dataComp['DATE'], format='%Y-%m-%d')


In [75]:
dataComp.set_index('DATE')

,TIPO,CONSUMO
DATE,,
2009-01-01,DOMESTICO MEDIO,20.0
2009-01-01,DOMESTICO MEDIO,NaN
2009-01-01,DOMESTICO RESIDENCIAL,NaN
2009-01-01,DOMESTICO MEDIO,NaN
2009-01-01,DOMESTICO RESIDENCIAL,20.0
...,...,...
2015-12-01,DOMESTICO MEDIO,22.0
2015-12-01,DOMESTICO MEDIO,3.0
2015-12-01,DOMESTICO MEDIO,8.0


In [11]:
#df.to_csv("waterconsumption.csv")

In [16]:

#RELLENANDO VALORES NAN POR LA MEDIA
dataComp=dataComp.fillna(20)
dataComp.index.name = 'DATE'
#CONVIRTIENDO EL INDICE DEL DATAFRAME EN TIPO DATETIME
#dataComp.index = pd.to_datetime(dataComp.index)
#type(dataComp.index)


In [17]:
dataComp.head()

,DATE,TIPO,CONSUMO
DATE,,,
0,2009-01-01,DOMESTICO MEDIO,20.0
1,2009-01-01,DOMESTICO MEDIO,20.0
2,2009-01-01,DOMESTICO RESIDENCIAL,20.0
3,2009-01-01,DOMESTICO MEDIO,20.0
4,2009-01-01,DOMESTICO RESIDENCIAL,20.0


In [13]:
y=dataComp.iloc[: , -1]
dataset=dataComp.iloc[:, :-1]
x=dataset
values = dataset.values
x

,DATE,TIPO
0,2009-01-01,DOMESTICO MEDIO
1,2009-01-01,DOMESTICO MEDIO
2,2009-01-01,DOMESTICO RESIDENCIAL
3,2009-01-01,DOMESTICO MEDIO
4,2009-01-01,DOMESTICO RESIDENCIAL
...,...,...
419995,2015-12-01,DOMESTICO MEDIO
419996,2015-12-01,DOMESTICO MEDIO
419997,2015-12-01,DOMESTICO MEDIO
419998,2015-12-01,DOMESTICO RESIDENCIAL


Grafica del consumo de agua de un usuario

In [14]:
# asegurarse que todo esta en formato float
values = values.astype('float32')
# normalzación
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled

ValueError: ignored

In [ ]:
#red neuronal estandar
#X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=.2)

numregistros=scaled.shape[0]
print(numregistros)

# split en conjuntos de train y test
n_train = int(round(numregistros*0.8,0)) #80% para entrenamiento
train_X = scaled[:n_train, :]
test_X = scaled[n_train:, :]
train_y = y[:n_train]
test_y = y[n_train:]

# reshape entrada a 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# Diseño de la red recurrente LSTM
model = Sequential()
model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(9))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# entrenar red
history = model.fit(train_X, train_y, epochs=1000, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
#FUNCION PARA ESCOGER UN REGISTRO DENTRO DEL DATAFRAME PARA LAS PRUEBAS DE VALIDACION
def selectRabbit(user):
  xnew = df.iloc[user:user+1 , 0:999]
  return xnew


In [ ]:
#FUNCION PARA BUSCAR EL VALOR REAL DE CONSUMO DENTRO DEL DATAFRAME
def searchForReal(user):
  real=xnew = df.iloc[user ,-1]
  return real

In [ ]:
#FUNCION PARA GRAFICAR LOS RESULTADOS
def graphResults(2):
  row = df.iloc[user:user+1 , 0:999]

In [ ]:
def prediccion(user):
    xnew=selectRabbit(user)
    Xnew= scaler.transform(xnew)
    Xnew = Xnew.reshape((Xnew.shape[0], 1, Xnew.shape[1]))
    pred= model.predict(Xnew)[0]
    return pred
    
pred=prediccion(83)
print('PREDICCION DE CONSUMO: ',pred)
print('USUARIO CONSUMO REAL :  [',searchForReal(83),']')

PRUEBAS PARA LA BINARIZACION DE LOS NOMBRES

In [ ]:
#TRYING TO TRANSPOSE DATA CONSUMPTION WITH NAMES OF CLIENTS
data= pd.read_csv('/content/AguaH.csv')
dataset= data.iloc[0:10,:]
dataset.shape
#ESTE CODIGO SIRVE UNICAMENTE PARA LA INTEGRACION DE LOS NOMBRES EN EL DATAFRAME
col_list = ["Name"]
names = pd.read_csv("/content/StateNames.csv", usecols=col_list)
names = names.iloc[0:10 , :]
#ELIMINAMOS LA COLUMNA DE LOS NOMBRE YA QUE SE BINARIZARON
#df = df.iloc[: , 1:]
print("Names shape ",names.shape)

In [ ]:
#AJUSTE DEL FORMATO DE LAS FECHAS PARA LA TRANSFORMACION A TIPO DATETIME
lookup = {'ENE':'01','FEB':'02','MAR':'03','ABR':'04','MAY':'05','JUN':'06','JUL':'07','AGO':'08','SEP':'09','OCT':'10','NOV':'11','DIC':'12'}
clist=[]
for col in dataset.columns[5:]:
    col = col[4:].split('_')
    clist.append('20'+col[1]+'-'+lookup[col[0]]+'-01')

columns = ['LANDUSE_TYPE','USER','PIPE DIAM','VENDOR','JAN16']+clist
dataset.columns=columns


In [ ]:
dataset = dataset.drop(columns=['LANDUSE_TYPE','PIPE DIAM','VENDOR','JAN16'])



In [ ]:
rows_list = []


In [ ]:
dataComp = pd.DataFrame(columns=('DATE','TIPO', 'CONSUMO'))
for column in range(1,len(dataset.columns),1):
  selection=dataset.iloc[:,column]
  for row in range(len(selection)):
    df2=[dataset.columns[column],dataset.iloc[row,0],selection.iloc[row]]
    print(df2)
    rows_list.append(df2)
    

In [ ]:
dataComp = pd.DataFrame(rows_list)               


In [ ]:
dataComp.shape

In [ ]:
dataComp.tail(100)


In [ ]:
dataset.tail()

In [ ]:
dataComp = pd.DataFrame(columns=('DATE','TIPO', 'CONSUMO'))
for column in range(2,len(dataset.columns),1):
  selection=dataset.iloc[:,column]
  for row in range(len(selection)):
    for iter in range(1,2,1):
      df2=[data.columns[iter],data.iloc[row,0],data.iloc[row,iter]]
      dataComp.loc[row]=df2


In [ ]:

  print()

In [ ]:
dataComp.head()

In [ ]:
datas

In [ ]:
names.index = dataset.index
dataset.insert(loc=1, column='Names', value=names)


In [ ]:
dataset=dataset.transpose()



In [ ]:
dataset.index.name = 'date'
dataset


In [ ]:
dataset.set_index(['date', 'Names']).unstack()


In [ ]:
data.set_index(['date', 'Names']).unstack()


In [ ]:
one_hot = pd.get_dummies(dataset['Names'])
dataset = dataset.drop('Names',axis = 1)
dataset = dataset.join(one_hot)


In [ ]:
dataset